# 한국 대형주 모멘텀 전략

## Basic Setting

In [40]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request as rq
from tqdm import tqdm

In [3]:
import yfinance as yf
from pandas_datareader import data as pdr
pd.core.common_is_list_likr = pd.api.types.is_list_like
yf.pdr_override()

In [4]:
import datetime
from datetime import timedelta

In [6]:
sample_code = '005930'
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN='

## Get Market Value

In [11]:
snap_html = rq.urlopen(SNAP_URL.format(sample_code)).read()
snap_soup = BeautifulSoup(snap_html, 'html.parser')

In [19]:
mv_cells = snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td')
market_value = float(mv_cells[6].string.replace(',',''))

## Get 12-month Price

In [23]:
end_date = datetime.date.today()
a_year = timedelta(days=365)
start_date = end_date - a_year

In [25]:
yf_code = sample_code + '.KS'

samsung_mk = pdr.get_data_yahoo(yf_code, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [27]:
samsung_mk.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-02-26,82800.0,83400.0,82000.0,82500.0,81020.203125,38520800
2021-03-02,85100.0,85300.0,83000.0,83600.0,82100.476562,33498180
2021-03-03,83500.0,84000.0,82800.0,84000.0,82493.304688,19882132
2021-03-04,82600.0,83200.0,82200.0,82400.0,80922.000000,24348331
2021-03-05,81100.0,82600.0,81100.0,82100.0,80627.382812,20508971


In [28]:
samsung_mk.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-21,74300.0,74300.0,74300.0,74300.0,74300.0,0
2022-02-22,74300.0,74300.0,74300.0,74300.0,74300.0,0
2022-02-23,74300.0,74300.0,74300.0,74300.0,74300.0,0
2022-02-24,72300.0,72300.0,71300.0,71500.0,71500.0,15759283
2022-02-25,72100.0,72600.0,71900.0,71900.0,71900.0,13168484


## Calculate RoR

In [29]:
start_price = samsung_mk.iloc[0]['Adj Close']

In [30]:
end_price = samsung_mk.iloc[-1]['Adj Close']

In [31]:
ror = ((end_price / start_price) - 1) * 100

## Get All Stock Code

In [32]:
company = pd.read_csv('company.csv')

In [55]:
kospi_code = company[company['분류'] == '코스피']['종목코드']

In [34]:
code_list = company['종목코드'].dropna()

In [35]:
code_list

0       000155
1       00088K
2       010955
3       051915
4       071055
         ...  
2017    013030
2018    019550
2019    019590
2020    019570
2021    006920
Name: 종목코드, Length: 2021, dtype: object

## Create Screening Function

In [48]:
def crawler(li):
    # dict for result DataFrame 
    result = {}
    
    # set Data
    end_date = datetime.date.today()
    a_year = timedelta(days=365)
    start_date = end_date - a_year
    
    for code in tqdm(li):
        try:
            # set URLs
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN='
            
            # get company name
            name = company[company['종목코드'] == code]['회사명'].values[0]
            
            # get Market Value
            snap_html = rq.urlopen(SNAP_URL.format(code)).read()
            snap_soup = BeautifulSoup(snap_html, 'html.parser')
            
            mv_cells = snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td')
            market_value = float(mv_cells[6].string.replace(',',''))
            
            # get 12-month Price
            yf_code = code + '.KS'
            stock_mk = pdr.get_data_yahoo(yf_code, start_date, end_date)
            
            start_price = samsung_mk.iloc[0]['Adj Close']
            end_price = samsung_mk.iloc[-1]['Adj Close']
            
            # calculate ROR
            ror = ((end_price / start_price) - 1) * 100
            
            # put data into result dict
            result[name] = [code, market_value, ror]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
        
    # convert to DataFrame
    result = pd.DataFrame(result)
    
    # transposing
    result = result.transpose()
    
    # set columns
    column_names = ['Code', 'Market_Value', 'RoR']
    result.columns = column_names
    
    return result

In [56]:
sample_df = crawler(kospi_code)

  2%|▏         | 13/781 [00:01<01:08, 11.24it/s]

[*********************100%***********************]  1 of 1 completed

  2%|▏         | 15/781 [00:02<03:24,  3.75it/s]


[*********************100%***********************]  1 of 1 completed

  2%|▏         | 16/781 [00:04<06:33,  1.94it/s]

...

100%|█████████▉| 779/781 [13:58<00:02,  1.10s/it]


[*********************100%***********************]  1 of 1 completed

100%|█████████▉| 780/781 [13:59<00:01,  1.08s/it]


[*********************100%***********************]  1 of 1 completed

100%|██████████| 781/781 [14:00<00:00,  1.08s/it]

In [57]:
result_df

,Code,Market_Value,RoR
제일약품,271980,3882.0,-11.256702
카카오,035720,420988.0,-11.256702
오리온,271560,37322.0,-11.256702
미원에스씨,268280,9971.0,-11.256702
경동도시가스,267290,1645.0,-11.256702
...,...,...,...
하이록코리아,013030,2348.0,-11.256702
SBI인베스트먼트,019550,2326.0,-11.256702
엠벤처투자,019590,1076.0,-11.256702
제미니투자,019570,616.0,-11.256702


## Screening

In [59]:
# Top 2000 MV
result_df = result_df.sort_values(by='Market_Value', ascending=False).head(200)

In [60]:
result_df

,Code,Market_Value,RoR
삼성전자,005930,4836202.0,-11.256702
SK하이닉스,000660,895443.0,-11.256702
NAVER,035420,515934.0,-11.256702
삼성바이오로직스,207940,508809.0,-11.256702
현대자동차,005380,427882.0,-11.256702
...,...,...,...
종근당,185750,11532.0,-11.256702
에스엘,005850,11472.0,-11.256702
코스모신소재,005070,11448.0,-11.256702
녹십자홀딩스,005250,11344.0,-11.256702


In [62]:
result_df[result_df['RoR'] > 0]

,Code,Market_Value,RoR
